In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hdbscan, numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score
from hdbscan.validity import validity_index

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the symptoms from the second file
with open('drive/My Drive/Projects/LSE-Symptomatology/Dataset/symptom_related_lexicon.txt', 'r') as file:
    symptoms_text = file.read()

In [ ]:
# Split the text by commas to get a list of symptoms and preprocess each symptom
all_symptoms_from_txt = [symptom.lower().strip() for symptom in symptoms_text.split(',')]
len(all_symptoms_from_txt)

31949

In [ ]:
# Remove symptoms named "PCOS" or "pcos"
all_symptoms_from_txt = [symptom for symptom in all_symptoms_from_txt if "pcos" not in symptom.lower()]

In [ ]:

# Count the number of symptoms after removal
number_of_symptoms_after_removal = len(all_symptoms_from_txt)
number_of_symptoms_after_removal

30478

In [ ]:
# Remove symptoms named "none"
all_symptoms_from_txt = [symptom for symptom in all_symptoms_from_txt if "none" not in symptom.lower()]
# Count the number of symptoms after removal
number_of_symptoms_after_removal = len(all_symptoms_from_txt)
number_of_symptoms_after_removal

30370

In [ ]:
import torch

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
from sentence_transformers import SentenceTransformer
with open('BioBERT.txt', 'r') as f:
    model_name = f.read().strip()
model = SentenceTransformer(model_name)
emb = model.encode(
    all_symptoms_from_txt,
    convert_to_numpy=True,
    normalize_embeddings=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install hdbscan

In [ ]:
def try_hdbscan(embeddings, mcs, ms):

    clusterer = hdbscan.HDBSCAN(
    min_cluster_size=mcs,
    min_samples=ms,
    metric='euclidean',
    cluster_selection_method='eom'
)

    labels = clusterer.fit_predict(embeddings)

    core_mask = labels >= 0
    if core_mask.sum() < 2:
        return None

    X_core = embeddings[core_mask].astype(np.float64)
    dbcv  = validity_index(X_core, labels[core_mask],metric='cosine')



    return {
        "min_cluster_size": mcs,
        "min_samples":       ms,
        "n_clusters":       len(np.unique(labels[core_mask])),
        "noise_frac":       1 - core_mask.mean(),
        "dbcv":             dbcv
    }

In [ ]:
grid = [(mcs, ms)
        for mcs in range(50, 101, 5)
        for ms  in (mcs, max(5, mcs//2))
        if ms != mcs or mcs > 5]          # ms = mcs or (mcs ≥5)

results = []
for mcs, ms in grid:
    out = try_hdbscan(emb, mcs, ms)
    if out is not None:
        results.append(out)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/usr/local/lib/python3.11/dist-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/usr/local/lib/python3.11/dist-packages/hdbscan/validity.py:30: RuntimeWarning: overflow encountered in power
  distance_matrix[distance_matrix != 0] = (1.0 / distance_matrix[
/usr/local/lib/python3.11/dist-packages/hdbscan/validity.py:30: RuntimeWarning: ov

In [ ]:
results

[{'min_cluster_size': 50,
  'min_samples': 50,
  'n_clusters': 94,
  'noise_frac': np.float64(0.5155416529469872),
  'dbcv': np.float64(0.4733873688786261)},
 {'min_cluster_size': 50,
  'min_samples': 25,
  'n_clusters': 107,
  'noise_frac': np.float64(0.4781692459664142),
  'dbcv': np.float64(0.48638800041351965)},
 {'min_cluster_size': 55,
  'min_samples': 55,
  'n_clusters': 81,
  'noise_frac': np.float64(0.5281527823510043),
  'dbcv': np.float64(0.4872163889101923)},
 {'min_cluster_size': 55,
  'min_samples': 27,
  'n_clusters': 94,
  'noise_frac': np.float64(0.4811326967402042),
  'dbcv': np.float64(0.4464003439824306)},
 {'min_cluster_size': 60,
  'min_samples': 60,
  'n_clusters': 73,
  'noise_frac': np.float64(0.5474481396114587),
  'dbcv': np.float64(0.48532082530096243)},
 {'min_cluster_size': 60,
  'min_samples': 30,
  'n_clusters': 87,
  'noise_frac': np.float64(0.4935133355284821),
  'dbcv': np.float64(0.451768048438387)},
 {'min_cluster_size': 65,
  'min_samples': 65,
  '

In [ ]:
from hdbscan import HDBSCAN
from collections import defaultdict
import numpy as np

def get_hdbscan_clusters(embeddings, items=None):
    clusterer = HDBSCAN(
        min_cluster_size=100,
        min_samples=50,
        metric='euclidean',
        cluster_selection_method='eom'
    )
    labels = clusterer.fit_predict(embeddings)
    print(f"Number of labels: {labels}")

    core_mask = labels >= 0
    if core_mask.sum() < 2:
        print("Too few core points. Skipping.")
        return None

    # number of clusters and noise
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_frac = np.sum(labels == -1) / len(labels)

    print(f"Number of clusters: {n_clusters}")
    print(f"Noise fraction: {noise_frac:.4f}")




In [ ]:
from collections import Counter

# clustering
result = get_hdbscan_clusters(emb, items=all_symptoms_from_txt)
if result is None:
    exit()

labels = result["labels"]
symptoms = all_symptoms_from_txt

# count frequencies
cluster_frequencies = Counter(labels)
clusters = {i: [] for i in set(labels)}

for symptom, label in zip(symptoms, labels):
    clusters[label].append(symptom)

# assign cluster names using most frequent symptom
cluster_names = {}
for cluster, symptom_list in clusters.items():
    most_common_symptom = Counter(symptom_list).most_common(1)[0][0]
    cluster_names[cluster] = most_common_symptom

# Print
for cluster, symptom_list in clusters.items():
    print(f"Cluster {cluster} (named '{cluster_names[cluster]}'): Frequency = {cluster_frequencies[cluster]}")
    print(", ".join(symptom_list))
    print("\n")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Number of labels: [37  3 -1 ... -1 -1  8]
Number of clusters: 50
Noise fraction: 0.5385
Cluster 0 (named 'high cholesterol'): Frequency = 114
high cholesterol, high cholesterol, cholesterol on edge, high lh, high lh/fsh ratio, high cholesterol, high lh levels, high cholesterol, high cholesterol, high cholesterol, high cholesterol, high cholesterol, high triglycerides, high cholesterol, high lh:fsh ratio, high cholesterol, high lh levels, high triglycerides, high lh levels, cholesterol levels, high lh, abnormal hdl/ldl cholesterol levels, high triglyceride levels, high lh levels, elevated cholesterol, high lh, high cholesterol, high triglycerides, high ldl cholesterol, cholesterol, high triglycerides, high lh/fsh ratio, elevated cholesterol, high triglycerides, high cholesterol, high cholesterol, high triglycerides, high lh levels, high lh/fsh ratio, high lh levels, high cholesterol, high cholesterol, high lh levels, cholesterol, elevated ldl cholesterol, abnormal cholesterol and trigly